## Simulation of Polymer Dynamics

This Jupyter Notebook documents the workflow for simulating polymer dynamics using HOOMD-blue. The simulation involves creating polymers, adding them to a simulation box, running the simulation, and analyzing the results. The key steps are outlined below:

1. **Import Libraries**: Import necessary libraries and modules.
2. **Define Functions**: Define functions for creating polymers and adding them to the simulation.
3. **Run Simulations**: Execute the simulation with specified parameters.
4. **Analyze Data**: Analyze the simulation data to extract meaningful insights.
5. **Plot Results**: Visualize the results using various plots.

Each section of the notebook contains detailed explanations and code snippets to guide you through the process.

### Import Libraries

In [ ]:
import numpy as np
import hoomd
import gsd
import random
import gsd.hoomd
import os
import freud
import os
import pickle
from datetime import date
import matplotlib.pyplot as plt
today = str(date.today())

### Define Functions

##### Create simulations and run with transcription dynamics

In [ ]:
def find_least_dense_area(snap,grid_size=10):
    """
    Find the least dense area in the simulation box
    Args:
        snap: hoomd.Snapshot object
        grid_size: size of the grid for histogram
    Returns:
        tuple: coordinates of the center of the least dense area
        float: minimum density value
    """
    # get the positions of the particles
    positions = snap.particles.position
    box=snap.configuration.box
    Lx,Ly,Lz=box[:3]

    # create a 3D histogram of the positions
    x_bins=np.linspace(-Lx/2,Lx/2,grid_size+1)
    y_bins=np.linspace(-Ly/2,Ly/2,grid_size+1)
    z_bins=np.linspace(-Lz/2,Lz/2,grid_size+1)

    hist,edges=np.histogramdd(positions,bins=(x_bins,y_bins,z_bins))
    min_density=np.min(hist)

    # find the center of the least dense area
    x_min,y_min,z_min=np.unravel_index(np.argmin(hist),hist.shape)
    x_center=(x_bins[x_min]+x_bins[x_min+1])/2
    y_center=(y_bins[y_min]+y_bins[y_min+1])/2
    z_center=(z_bins[z_min]+z_bins[z_min+1])/2

    return (x_center,y_center,z_center), min_density

def euclidean_distance(point1, point2):
    """
    Calculate the Euclidean distance between two points in 3D space.
    
    Args:
        point1 (tuple): The (x, y, z) coordinates of the first point.
        point2 (tuple): The (x, y, z) coordinates of the second point.
    
    Returns:
        float: The Euclidean distance between the two points.
    """
    return np.sqrt(np.sum((np.array(point1) - np.array(point2)) ** 2))

def check_overlaps(polymer_dict):
    """
    Check for overlaps between polymers in the given dictionary.
    
    Args:
        polymer_dict (dict): A dictionary where keys are polymer IDs and values are tuples of x, y, z coordinates.
    
    Returns:
        None
    """
    overlapping_points = []

    # Check for overlapping points or points with distance < 1 between polymers
    for polymer_id, (x, y, z) in polymer_dict.items():
        points = list(zip(x, y, z))
        num_points = len(points)
        for i in range(num_points):
            for j in range(i + 1, num_points):
                if euclidean_distance(points[i], points[j]) < 1:
                    overlapping_points.append((polymer_id, points[i], points[j]))

    if overlapping_points:
        print("Overlapping points or points with distance < 1 found:")
        for polymer_id, point1, point2 in overlapping_points:
            print(f"Polymer {polymer_id}: {point1} and {point2}")
    else:
        print("No overlapping points or points with distance < 1 found.")


def choose_seeds(NP,L,box_length,spacing,current_positions=[],choose=True,seedfill=0):
    """
    Choose seed positions for polymers in the simulation box.
    
    Args:
        NP (int): Number of polymers.
        L (int): Length of each polymer.
        box_length (float): Length of the simulation box.
        spacing (float): Region of replication in simulation box.
        current_positions (list): List of current polymer positions.
        choose (bool): Whether to choose new seed positions.
        seedfill (int): Seed value for random number generator.
    
    Returns:
        int: Seed value for random number generator.
        int: Seed value for molecular dynamics.
    """

    if choose:
        seed=random.randint(0,2**32-1)
        mdseed=random.randint(0,65535)
    else:
        seed=seedfill

 
    trajdict=multiple_walks(NP,L,box_length,spacing,seed,current_positions)
    if trajdict==None:
        while trajdict is None:
            seed=random.randint(0,2**32-1)
            trajdict=multiple_walks(NP,L,box_length,spacing,seed,current_positions)

    return seed,mdseed

def surrounding_check(x_p,y_p,z_p, b,polymer_check):
    """
    Check if the new position is surrounded by other polymer positions within a distance of 1 unit.
    
    Args:
        x_p (int): x-coordinate of the position to check.
        y_p (int): y-coordinate of the position to check.
        z_p (int): z-coordinate of the position to check.
        b (int): Distance between particles.
        polymer_check (set): Set of all polymer positions.
    
    Returns:
        bool: True if the position is surrounded by other polymer positions, False otherwise.
    """
    possible_moves = [(b, 0, 0), (-b, 0, 0), (0, b, 0), (0, -b, 0), (0, 0, b), (0, 0, -b)]
    for move in possible_moves:
        new_x = x_p + move[0]
        new_y = y_p + move[1]
        new_z = z_p + move[2]
        #dist= np.sqrt((new_x-x_p)**2+(new_y-y_p)**2+(new_z-z_p)**2)
        if (new_x, new_y, new_z) not in polymer_check: # and dist>=1:
            # Found a move that is not occupied, so the position is not completely surrounded
            return False
    # All possible moves are occupied, so the position is completely surrounded
    return True


def surrounding_current(x_p,y_p,z_p,set_positions):
    """
    Check if the current position is within 1 unit of any existing position.
    
    Args:
        x_p (int): x-coordinate of the current position.
        y_p (int): y-coordinate of the current position.
        z_p (int): z-coordinate of the current position.
        set_positions (set): Set of all existing polymer positions.
    
    Returns:
        bool: True if the current position is within 1 unit of any existing position, False otherwise.
    """
    for pos in set_positions:
        differences=np.linalg.norm(pos-np.array([x_p,y_p,z_p]))
        if (x_p,y_p,z_p) in set_positions or differences<1:
            # The new position is within 1 unit of an existing position
            return True
    # There is no existing position within 1 unit of the new position
    return False


def multiple_walks(NP, L, box_length,spacing,seed, current_positions=[]):  
    """
    Generate multiple random walks for polymers in a simulation box.
    
    Args:
        NP (int): Number of polymers.
        L (int): Length of each polymer.
        box_length (float): Length of the simulation box.
        spacing (float): Region of replication in simulation box.
        seed (int): Seed value for random number generator.
        current_positions (list): List of current polymer positions.
    
    Returns:
        dict: A dictionary where keys are polymer IDs and values are tuples of x, y, z coordinates.
    """

    # Set the random seed
    random.seed(seed)
    np.random.seed(seed)

    # Initialize the polymer dictionary
    polymer_dict={}
    b=1         
    polymer_check=set()  
    
    # Add the current positions to the set if they exist
    set_positions=set()
    if len(current_positions)>0:
        for x,y,z in current_positions:
            set_positions.add((x,y,z))
    else:
        set_positions=set()
    
    # Generate the random walk for each polymer
    for polymer_id in range(NP):
        # Initialize the polymer coordinates
        start_box_size=int(box_length/spacing) 
        x=np.zeros(L)
        y=np.zeros(L)
        z=np.zeros(L)
        fail_starts=set()
        start=False

        # Choose a random starting position
        while not start:
            available_starts=[(x,y,z) for x in range(-start_box_size, start_box_size + 1) 
                            for y in range(-start_box_size ,start_box_size +1)
                            for z in range(-start_box_size,start_box_size+1)
                            if (x,y,z) not in fail_starts]
            
            start_position=available_starts[np.random.randint(len(available_starts))]
            x[0],y[0],z[0]=start_position

            # Check if the starting position is valid
            if (x[0],y[0],z[0]) not in polymer_check and not surrounding_current(x[0],y[0],z[0],set_positions):
                polymer_check.add((x[0],y[0],z[0]))
                start=True
            else:
                fail_starts.add(start_position)
    
        # Generate the random walk for the polymer
        possible_steps=[(b,0,0),(-b,0,0),(0,b,0),(0,-b,0),(0,0,b),(0,0,-b)] #5,1,0,3 
        for i in range(1, L):
            failed_steps=set()
            step_taken = False
            # Try to take a step until a valid step is taken
            while not step_taken:
                new_x, new_y, new_z = x[i - 1], y[i - 1], z[i - 1]
                available_steps=[step for step in possible_steps if step not in failed_steps] #this index is changing as fail steps occur
                # Randomly generate the step directions
                if len(available_steps)==0:
                    return None
                step_index=np.random.randint(len(available_steps))
                move=available_steps[step_index]
                # Calculate potential new position
                new_x += move[0]
                new_y += move[1]
                new_z += move[2]
                # Check if the new position is already occupied
                if ((new_x, new_y, new_z) not in polymer_check and
                    not surrounding_current(new_x, new_y, new_z, set_positions) and
                    not surrounding_check(new_x, new_y, new_z, b, polymer_check) and
                    np.abs(new_x) < box_length / 2 and
                    np.abs(new_y) < box_length / 2 and
                    np.abs(new_z) < box_length / 2):
                    x[i], y[i], z[i] = new_x, new_y, new_z
                    polymer_check.add((new_x, new_y, new_z))
                    step_taken=True 
                else: 
                    failed_steps.add(move)
        # Add the polymer to the dictionary
        polymer_dict[polymer_id]=(x,y,z)
    return polymer_dict


In [ ]:
class AddPolymer(hoomd.custom.Action):
    """Add a polymer to the simulation box
    
    Args:
        sim (hoomd.Simulation): The simulation object.
        L (int): The length of the polymer.
        LE (int): The number of monomers of type E.
        LM (int): The number of monomers of type M.
        box_length (float): The length of the simulation box.
        spacing (float): Region of replication in simulation box.
        seed (int): The seed for the random number generator.
        monomer_types (list): The list of monomer types.
        countcheck (int): The number of polymers to check for before adding more.
    """
    def __init__(self, sim, L, LE,LM,box_length,spacing,seed,monomer_types=['A','B','C'],countcheck=50):
        super().__init__()
        self.sim = sim
        self.L = L
        self.LE = LE
        self.LM = LM
        self.box_length = box_length
        self.spacing = spacing
        self.seed = seed
        self.monomer_types = monomer_types
        self.countcheck=countcheck

    def act(self, timestep):
        L = self.L
        snapshot=self.sim.state.get_snapshot()
        if self.sim.device.communicator.rank == 0:
            # Add new particles to the snapshot
            N = self.L
            new_snapshot = snapshot
            current_pos = snapshot.particles.position
            num_poly=len(current_pos)/N

            # Check if the number of polymers is less than the countcheck
            if num_poly < self.countcheck:
                seed,mdseed = choose_seeds(1, L, self.box_length,self.spacing, current_positions=current_pos, choose=True, seedfill=self.seed)
                polymer_dict = multiple_walks(1, L, self.box_length,self.spacing,seed, current_positions=current_pos)
                new_snapshot.particles.N += N
                positions=[]

                # Add the new polymer to the snapshot
                for polymer in polymer_dict:
                    x,y,z=polymer_dict[polymer]
                    positions.extend([(xi,yi,zi) for xi,yi,zi in zip(x,y,z)])
                new_snapshot.particles.position[-N:] = positions
                new_snapshot.particles.velocity[-N:] = np.random.normal(0, 1, (N, 3))
                new_snapshot.particles.typeid[-N:] = [1]*self.LE + [0]*self.LM + [2]*self.LE
                new_snapshot.particles.mass[-N:] = 1.0  
                new_snapshot.particles.diameter[-N:] = 1.0
                new_snapshot.bonds.N += (L - 1)
                new_snapshot.bonds.types = ['A-A']
                new_snapshot.bonds.typeid[-(L-1):]=[0]*(L-1)
                bond_groups = []
                start_index = new_snapshot.particles.N - N
                
                # Add bonds between the particles in the polymer
                for i in range(start_index, start_index + L - 1):
                    bond_groups.append([i, i + 1])
                new_snapshot.bonds.group[-(L-1):] = bond_groups
                harmonic = hoomd.md.bond.Harmonic()
                harmonic.params['A-A']=dict(k=100,r0=1.0)
                
                # Define the Lennard-Jones potential
                #lennard jones
                #RNA-RNA
                lj = hoomd.md.pair.LJ(nlist=hoomd.md.nlist.Cell(buffer=0.4),default_r_cut=2.5,mode='shift')
                lj.params[('A','A')] = dict(epsilon=3,sigma=1)
                lj.params[('B','B')] = dict(epsilon=0.1,sigma=1)
                lj.params[('C','C')] = dict(epsilon=0.1,sigma=1)
                lj.params[('A','B')] = dict(epsilon=0.1,sigma=1)
                lj.params[('A','C')] = dict(epsilon=0.1,sigma=1)
                lj.params[('B','C')] = dict(epsilon=0.1,sigma=1)
                force=[harmonic,lj]


                # Set the forces
                self.sim.operations.integrator.forces = force
                # Set the new snapshot with bonds as the current state
                self.sim.state.set_snapshot(new_snapshot)


##### Running simulations functions

In [ ]:
#make initial frame
def createframe(N,L,LE,LM,box_size,spacing,IS_file):
    """Create the initial frame for the simulation

    Args:
        N (int): Number of polymers.
        L (int): Length of each polymer (ie. number of monomers).
        LE (int): Number of monomers of type 3'/5'.
        LM (int): Number of monomers of type M.
        box_size (float): Size of the simulation box.
        spacing (float): SRegion of replication in simulation box.
        IS_file (str): Initial snapshot file name.

    Returns:
        tuple: Seeds for the polymer and molecular dynamics.
    """
    
    # Choose seeds for the polymer
    seeds,mdseed=choose_seeds(N,L,box_size,spacing,current_positions=[])
    # Generate the polymer dictionary
    polymer_dict = multiple_walks(N,L,box_size,spacing,seeds,current_positions=[])
    #Create the initial frame
    frame=gsd.hoomd.Frame()
    frame.particles.N=L
    positions=[]
    for polymer_id in polymer_dict:
        x,y,z=polymer_dict[polymer_id]
        positions.extend([(xi,yi,zi) for xi,yi,zi in zip(x,y,z)])
    frame.particles.position=positions
    frame.particles.types=['A','B','C']
    # Define bonds
    frame.bonds.N = (L - 1) * N
    frame.bonds.types = ['A-A']
    frame.bonds.typeid = [0] * frame.bonds.N
    bond_groups = []
    for n in range(N):
        start_index = n * L
        for i in range(start_index, start_index + L - 1):
            bond_groups.append([i, i + 1])
    frame.bonds.group = bond_groups

    #Define particle types
    frame.particles.typeid=[1]*LE+[0]*LM+[2]*LE
    frame.particles.mass=np.array([1.0]*L)
    frame.particles.diameter=np.array([1.0]*L)
    frame.configuration.box=np.array([box_size,box_size,box_size,0,0,0])

    with gsd.hoomd.open(IS_file, 'w') as f:
        f.append(frame)

    return seeds,mdseed


 
def add_forces(trig,Mep,Eep,EMep,seeds,mdseed,IS_file,LE,LM,L,box_size,spacing,run):
        """Add forces to the simulation

        Args:
            trig (int): Trigger period for adding polymers.
            Mep (float): Lennard-Jones epsilon parameter for middle monomer interactions.
            Eep (float): Lennard-Jones epsilon parameter for 5'/3' monomer interactions.
            EMep (float): Lennard-Jones epsilon parameter for 5'/3'-middle monomer interactions.
            seeds (list): Seeds for the polymer.
            mdseed (int): Seed for the molecular dynamics.
            IS_file (str): Initial frame file.
            LE (int): Number of monomers of type 3'/5'.
            LM (int): Number of monomers of type M.
            L (int): Length of each polymer (ie. number of monomers).
            box_size (float): Size of the simulation box.
            spacing (float): Region of replication in simulation box.
            run (int): Run number for the simulation.
        """

        #harmonic bond
        harmonic = hoomd.md.bond.Harmonic()
        harmonic.params['A-A']=dict(k=100,r0=1.05)

        #lennard jones
        #RNA-RNA
        lj = hoomd.md.pair.LJ(nlist=hoomd.md.nlist.Cell(buffer=0.4),default_r_cut=2.5,mode='shift')
        lj.params[('A','A')] = dict(epsilon=Mep,sigma=1)
        lj.params[('B','B')] = dict(epsilon=Eep,sigma=1)
        lj.params[('C','C')] = dict(epsilon=Eep,sigma=1)
        lj.params[('A','B')] = dict(epsilon=EMep,sigma=1)
        lj.params[('A','C')] = dict(epsilon=EMep,sigma=1)
        lj.params[('B','C')] = dict(epsilon=EMep,sigma=1)
        force=[harmonic,lj]

        #initialize simulation
        device=hoomd.device.CPU()
        sim=hoomd.Simulation(device=device,seed=mdseed)
        sim.create_state_from_gsd(filename=IS_file)
        langevin = hoomd.md.methods.Langevin(filter=hoomd.filter.All(),kT=1.0)
        integrator=hoomd.md.Integrator(dt=1e-2,methods=[langevin],forces=force)
        sim.operations.integrator=integrator

        #use AddPolymer class to add polymers over steps
        add_polymer=AddPolymer(sim, L, LE, LM, box_size, spacing, seeds)
        trigger=hoomd.trigger.Periodic(int(trig))
        custom_op=hoomd.update.CustomUpdater(trigger=trigger,action=add_polymer)
        sim.operations.updaters.append(custom_op)

        #add logger
        thermo=hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())
        sim.operations.computes.append(thermo)
        logger=hoomd.logging.Logger()
        logger.add(sim,quantities=['timestep','tps','walltime'])
        logger.add(thermo)

        #make trajectory file
        name=f'Trajectory_trigger_{trig}_sp_{spacing}_Mep_3_Ep01_limit50_run_{run}.gsd'
        path=os.path.dirname('Trajectories/Limit50_L45/')
        os.makedirs(path,exist_ok=True)
        outpath=os.path.join(path,name)

        #log gsd
        gsd_writer=hoomd.write.GSD(trigger=hoomd.trigger.Periodic(100), ##ENTER PERIOD LOGGING FOR SIMULATION
                filename=outpath,
                filter=hoomd.filter.All(),
                mode='wb',
                logger=logger)

        sim.operations.writers.append(gsd_writer)
        sim.run(5e4) ## ENTER TIME FOR SIMULATION
        gsd_writer.flush()

##### Analysis Functions

In [ ]:
class CumulativeAverage:
    """
    A class to calculate the cumulative average of a list, numpy array, or dictionary of values.
    """
    def __init__(self):
        """
        Initialize the CumulativeAverage class.
        """
        self.numbers = []
        self.dict={}
        self.current_average = None
        self.cumulative_averages = []

    def add_value(self, new_value):
        """Adding new data to list"""

        #If new value is a float
        if isinstance(new_value, (int,float)):
            self.numbers.append(new_value)
            sum_list=0
            if self.current_average is None:
                self.current_average = new_value
            else:
                for i in range(len(self.numbers)):
                    sum_list+=self.numbers[i]
                self.current_average = sum_list/(len(self.numbers))
        
        #If new value is an array
        elif isinstance(new_value, (list,np.ndarray)):
            new_value=np.array(new_value)
            self.numbers.append(new_value)
            if self.current_average is None:
                self.current_average = new_value
            else:
                sum_list=0
                for i in range(len(self.numbers)):
                    sum_list+=self.numbers[i]
                self.current_average = sum_list/(len(self.numbers))

        #If new value is a dictionary
        elif isinstance(new_value, dict):
            for key, value in new_value.items():
                if key not in self.numbers:
                    self.dict[key] = []
                self.dict[key].append(value)
                if self.current_average is None:
                    self.current_average = {key: value}
                else:
                    sum_list=0
                    for i in range(len(self.dict[key])):
                        sum_list+=self.dict[key][i]
                    self.current_average[key] = sum_list/(len(self.dict[key]))
        self.cumulative_averages.append(self.current_average)
        return self.current_average
    
    #return cumulative average
    def get_cumulative_averages(self):
        """
        Get the cumulative averages.

        Returns:
            list: A list of cumulative averages.
        """
        return self.cumulative_averages
    


class DataAverager:
    """
    A class to average data from multiple dictionaries.
    """
    def __init__(self):
        """
        Initialize the DataAverager class.
        """
        self.data = {}
        self.file_count = {}

    def add_data(self, dictionary):
        """
        Add data from a dictionary to the DataAverager.

        Args:
            dictionary (dict): A dictionary where keys are data labels and values are lists or numpy arrays of data points.
        """
        for key,data in dictionary.items():
            if key not in self.data:
                self.data[key] = np.array(data)
                self.file_count[key] = 1
            else:
                self.data[key] += np.array(data)
                self.file_count[key] += 1

    def compute_average(self):
        """
        Compute the average of the data.

        Returns:
            dict: A dictionary where keys are data labels and values are the averaged data points.
        """
        averaged_data = {key: value / self.file_count[key] for key, value in self.data.items()}
        return averaged_data

def savedata(path,dataname,data):
    """
    Save data to a specified path using pickle.

    Args:
        path (str): The directory path where the data will be saved.
        dataname (str): The name of the file to save the data as.
        data (any): The data to be saved.
    """
    DA_path=path+dataname
    os.makedirs(os.path.dirname(DA_path), exist_ok=True)
    with open(DA_path, 'wb') as f:
        pickle.dump(data, f)

def cluster(gsd_file,L=45):
    """
    Analyze clusters in a GSD file.

    Args:
        gsd_file (str): The path to the GSD file to analyze.
        L (int, optional): The length of the polymers. Defaults to 45.

    Returns:
        dict: A dictionary containing the number of polymers, number of clusters, cluster mass, and time.
    """
    #Initialize properties to average and to append
    data={}
    num_clusters=CumulativeAverage()
    num_p=[]
    clustermass=CumulativeAverage()
    time=[]

    #Run through each frame
    with gsd.hoomd.open(gsd_file, 'r') as traj:
        for n,frame in enumerate(traj):
            
            positions = frame.particles.position
            box=frame.configuration.box

            #determine clusters and cluster properties
            cp= freud.cluster.Cluster()
            system=freud.AABBQuery(box,positions)
            cp.compute(system,neighbors={'r_max':2})  
            clp=freud.cluster.ClusterProperties()
            clp.compute(system,cp.cluster_idx)

            #store data to cumulative average
            num_particles = len(positions)
            num_polymers=num_particles//L
            num_p.append(num_polymers)
            num_clusters.add_value(float(cp.cluster_idx.max()))
            clustermass.add_value(np.max(clp.sizes)/L)
            time.append(frame.configuration.step)
        
        #get cumulative averages
        number_clust=num_clusters.get_cumulative_averages()
        number_mass=clustermass.get_cumulative_averages()
        data['num_polymers']=num_p
        data['num_clusters']=number_clust
        data['cluster_mass']=number_mass
        data['time']=time

    return data

##### Plotting Functions

In [ ]:
def render_simulation_snapshot(gsd_file,num_frames=1,L=45):
    """
    Render the last few frames of a simulation from a GSD file.

    Args:
        gsd_file (str): Path to the GSD file containing the simulation data.
        num_frames (int): Number of frames to render from the end of the simulation. Default is 1.
        L (int): Length of each polymer. Default is 45.
    """
    # Open the GSD file
    with gsd.hoomd.open(gsd_file, 'r') as traj:
        # Read the last frame
        for n,frame in enumerate(traj[-num_frames:]):
        
            # Extract particle positions
            box_size=frame.configuration.box[0]
            positions = frame.particles.position
            num_particles = len(positions)  
            num_polymer = num_particles//L
            

            # Create a color array
            colors = np.zeros(num_particles, dtype=int)
            for i in range(num_polymer):
                colors[i*L:(i+1)*L] = i
            
            # Create a colormap
            cmap = plt.get_cmap('viridis', num_polymer)
            # Create a 3D scatter plot of the particle positions
            fig = plt.figure(figsize=(8, 8))
            ax = fig.add_subplot(111, projection='3d')
            scatter = ax.scatter(positions[:, 0], positions[:, 1], positions[:, 2], c=colors, cmap=cmap, s=1)
            #ax.plot(positions[:, 0], positions[:, 1], positions[:, 2], 'k-', alpha=0.1)
            ax.set_xlim(-box_size/2, box_size/2)
            ax.set_ylim(-box_size/2, box_size/2)
            ax.set_zlim(-box_size/2, box_size/2)
            ax.set_xlabel('X Position')
            ax.set_ylabel('Y Position')
            ax.set_zlabel('Z Position')
            ax.set_title(f'Simulation Snapshot at Frame {n}')
            
            # Add a color bar
            cbar = plt.colorbar(scatter, ax=ax, ticks=range(num_polymer))
            cbar.set_label('Segment Index')
            plt.show()

def plot_across(fileset,outputfolder,save=False):
    """Plot the number of clusters, number of polymers, and average cluster size across different subregions sizes and trigger rates
    Args:
    - fileset: dictionary of files
    - outputfolder: list of output files
    - save: boolean to save the plots
    """
    
    i=0
    color=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d','#666666'] #set of colors

    #loop through each average set of data
    for name,file in fileset.items():
        #load data
        data=pickle.load(open(file,'rb'))
        time=data['time']
        number_clust=data['num_clusters']
        number_mass=data['cluster_mass']
        num_polymers=data['num_polymers']  

        #plot number of clusters in simulation
        plt.figure(1)
        plt.plot(time,number_clust,label=name,color=color[i])
        plt.xlabel('Simulation Time')
        plt.ylabel('Number of Clusters')
        plt.legend(loc=0)
        plt.title(f'Number of Clusters')
        if save:
            plt.savefig(outputfolder+'Number_of_Clusters.png',bbox_inches='tight',dpi=600)

        #plot average number of polymers in largest clusters
        plt.figure(2)
        plt.plot(time,number_mass,label=name,color=color[i])
        plt.xlabel('Simulation Time')
        plt.ylabel('Number of Polymer Chains')
        plt.title(f'Number of Polymer Chains per Largest Cluster')
        plt.legend(loc=0)
        if save:
            plt.savefig(outputfolder+'PolymerChains_in_LargestClusters.png',bbox_inches='tight',dpi=600)

        #plot total number of polymers in simulation
        plt.figure(3)
        plt.plot(time,num_polymers,label=name,color=color[i])
        plt.xlabel('Simulation Time')
        plt.ylabel('Number of Polymer Chains')
        plt.title(f'Number of Polymer Chains in Simulation')
        plt.legend(loc=0)
        i+=1
        if save:
            plt.savefig(outputfolder+'TotalNumber_of_Polymers.png',bbox_inches='tight',dpi=600)


    plt.show()



### Run Simulations

In [ ]:
#Parameters
N=1 #number of polymers (at beginning)
L=45 #length of polymer
LM=15 #number of monomers of type M
LE=15 #number of monomers of type E
box_size=150 #size of the box
spacing=[24,2] #subregion of box
Mep=3 #epsilon for A-A interaction
Eep=0.1 #epsilon for B-B and C-C interaction
EMep=0.1 #epsilon for A-B, A-C, and B-C interaction
runs=1 #number of runs
trig=[100,800] #trigger for adding polymers


#loop and run
for sp, tr in zip(spacing,trig):
        IS_file=f'Initial_Polymer_{sp}_L_{L}.gsd' #Making initial trajectory file for certain groups
        for run in range(runs):
            seeds,mdseed=createframe(N,L,LE,LM,box_size,sp,IS_file)
            add_forces(tr,Mep,Eep,EMep,seeds,mdseed,IS_file,LE,LM,L,box_size,sp,run)
            print(f'Run {run} completed with spacing {sp} and trigger {tr}')

### Analyze Data

##### Analyze trajectories and average over replicates

In [ ]:
#Define folder for trajectories
folder='Trajectories/Limit50_L45/'

#Define folder to store analyzed data
Output=f'Outputs/{today}/PolymerLength_45_largestcluster/'


#Loop through files, analyze, average
for sp,tr in zip(spacing,trig):
    #file name 
    filecheck=f'Trajectory_trigger_{tr}_sp_{sp}_Mep_3_Ep01_limit50'

    #initialize average
    filedata={}
    dataavg=DataAverager() 

    #average data
    for file in os.listdir(folder): 
        if file.endswith('.gsd') and filecheck in file:
            data=cluster(folder+file,L=45)
            filedata[file]=data
            dataavg.add_data(filedata[file])
    averagedata=dataavg.compute_average()
    #save data
    savedata(Output,filecheck,averagedata)

        

### Plot Results

##### Plot renders

In [ ]:
#Loop through each trajectory and plot
for file in os.listdir(folder):
    print(file)
    render_simulation_snapshot(folder+file)

##### Plot cluster analysis

In [ ]:
#Pull output folder of analyzed data
outputfolder=f'Outputs/{today}/PolymerLength_45_largestcluster/'
#Create a figures folder
figurefolder=f'Figures/{today}/PolymerLength_45_largestcluster/'
os.makedirs(figurefolder,exist_ok=True)

#loop throiugh files in loop and store data
files=[]
for file in os.listdir(outputfolder):
    files.append(outputfolder+file)
files.sort()

#create a dictionary describing features of each trajectory then plot
fileset={'Transcription Faster than Diffusion':files[0],'Transcription Slower than Diffusion':files[1]}
plot_across(fileset,figurefolder,save=True)